In [1]:
# activate project environment
import Pkg
if !haskey(Pkg.installed(), "AA228FinalProject")
    jenv = joinpath(dirname(@__FILE__()), ".") # this assumes the notebook is in the same dir
    # as the Project.toml file, which should be in top level dir of the project. 
    # Change accordingly if this is not the case.
    Pkg.activate(jenv)
end

"C:\\Users\\Ali\\documents\\stanford\\cs 238\\AA228FinalProject\\Project.toml"

In [2]:
# import necessary packages
using AA228FinalProject
using POMDPs
using POMDPPolicies
using BeliefUpdaters
using ParticleFilters
using POMDPSimulators
using Cairo
using Gtk
using Random
using Printf
using POMDPModels
using POMCPOW

┌ Info: Precompiling AA228FinalProject [fe2df5ea-4d44-4e5a-a895-9dbc87b19b37]
└ @ Base loading.jl:1192
┌ Info: Recompiling stale cache file C:\Users\Ali\.julia\compiled\v1.0\POMDPSimulators\i1HOp.ji for POMDPSimulators [e0d0a172-29c6-5d4e-96d0-f262df5d01fd]
└ @ Base loading.jl:1190
┌ Info: Recompiling stale cache file C:\Users\Ali\.julia\compiled\v1.0\POMDPModels\GHWgR.ji for POMDPModels [355abbd5-f08e-5560-ac9e-8b5f2592a0ca]
└ @ Base loading.jl:1190
┌ Info: Loading Cairo backend into Compose.jl
└ @ Compose C:\Users\Ali\.julia\packages\Compose\BYWXX\src\Compose.jl:161
┌ Warning: Package Compose does not have Cairo in its dependencies:
│ - If you have Compose checked out for development and have
│   added Cairo as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with Compose
│ Loading Cairo into Compose from project dependency, future warnings for Compose are suppressed.
└ @ nothing nothing:

In [6]:
# generate discrete action space
vlist = [0, 5]
omlist = [-1, 0, 1]
aspace = vec(collect(RoombaAct(v, om) for v in vlist, om in omlist))

# generate discrete state space
num_y_pts = 50
num_x_pts = 50
num_th_pts = 20    
sspace = DiscreteRoombaStateSpace(num_x_pts, num_y_pts, num_th_pts)

# initialize POMDP interface
sensor = Bumper()
config = 3 # 1, 2, or 3
m = RoombaPOMDP(sensor = sensor, mdp = RoombaMDP(config = config, aspace = aspace, sspace = sspace))

num_particles = 2000
resampler = BumperResampler(num_particles)

spf = SimpleParticleFilter(m, resampler)

v_noise_coefficient = 1.0
om_noise_coefficient = 0.25

belief_updater = RoombaParticleFilter(spf, v_noise_coefficient, om_noise_coefficient);

In [7]:
# specify POMCPOW solver
solver = POMCPOWSolver(criterion = MaxUCB(20.0))
planner = solve(solver, m)

POMCPOWPlanner{RoombaPOMDP{Bumper,Bool},POMCPOW.POWNodeFilter,MaxUCB,MCTS.RandomActionGenerator{MersenneTwister},BasicPOMCP.SolvedPORollout{RandomPolicy{MersenneTwister,RoombaPOMDP{Bumper,Bool},NothingUpdater},NothingUpdater,MersenneTwister},Int64,Float64,POMCPOWSolver}(POMCPOWSolver
  eps: Float64 0.01
  max_depth: Int64 9223372036854775807
  criterion: MaxUCB
  final_criterion: MaxQ MaxQ()
  tree_queries: Int64 1000
  max_time: Float64 Inf
  rng: MersenneTwister
  node_sr_belief_updater: POMCPOW.POWNodeFilter POMCPOW.POWNodeFilter()
  estimate_value: MCTS.RolloutEstimator
  enable_action_pw: Bool true
  check_repeat_obs: Bool true
  check_repeat_act: Bool true
  tree_in_info: Bool false
  alpha_observation: Float64 0.5
  k_observation: Float64 10.0
  alpha_action: Float64 0.5
  k_action: Float64 10.0
  init_V: Float64 0.0
  init_N: Int64 0
  next_action: MCTS.RandomActionGenerator{MersenneTwister}
  default_action: MCTS.ExceptionRethrow MCTS.ExceptionRethrow()
, RoombaPOMDP{Bumper,Bo

In [76]:
# define the heuristic policy as follows
struct MyHeuristic <: Policy
    # intentionally empty
end

# extract goal for heuristic controller
goal_x, goal_y = get_goal_xy(m)
print(get_goal_xy(m))

# define what the policy actually does in POMDPs.action
function POMDPs.action(p::MyHeuristic, s::RoombaState) 
    # compute the difference between our current heading and one that would
    # point to the goal
    x, y, th = s[1:3]
    ang_to_goal = atan(goal_y - y, goal_x - x)
    del_angle = wrap_to_pi(ang_to_goal - th)
    
    
    if abs(del_angle) < 0.25
        v, om = 5, 0
    else
        i = 1
        while i <= length(omlist) && omlist[i] < del_angle
            i += 1
        end

        if i > length(omlist)
            i -= 1
        end

        v, om = 0, omlist[i]
    end
    
    return RoombaAct(v, om)
end

p = MyHeuristic()

# specify POMCPsolver
solver = POMCPSolver(estimate_value = FORollout(p), c = 10)

planner = solve(solver, m)

[15.0, 0.0]

POMCPPlanner{RoombaPOMDP{Bumper,Bool},BasicPOMCP.SolvedFORollout{MyHeuristic,MersenneTwister},MersenneTwister}(POMCPSolver
  max_depth: Int64 20
  c: Float64 10.0
  tree_queries: Int64 1000
  max_time: Float64 Inf
  tree_in_info: Bool false
  default_action: ExceptionRethrow ExceptionRethrow()
  rng: MersenneTwister
  estimate_value: FORollout
, RoombaPOMDP{Bumper,Bool}(Bumper(), RoombaMDP{ContinuousRoombaStateSpace,Array{RoombaAct,1}}
  v_max: Float64 10.0
  om_max: Float64 1.0
  dt: Float64 0.5
  contact_pen: Float64 -1.0
  time_pen: Float64 -0.1
  goal_reward: Float64 10.0
  stairs_penalty: Float64 -10.0
  config: Int64 3
  sspace: ContinuousRoombaStateSpace ContinuousRoombaStateSpace()
  room: AA228FinalProject.Room
  aspace: Array{RoombaAct}((24,))
  _amap: Dict{RoombaAct,Int64}
), BasicPOMCP.SolvedFORollout{MyHeuristic,MersenneTwister}(MyHeuristic(), MersenneTwister(UInt32[0x00000002], Random.DSFMT.DSFMT_state(Int32[-455407987, 1073130395, -1496131950, 1073726245, -921510624, 107

In [10]:
# simulate our policy and print the reward
rollout_sim = RolloutSimulator(max_steps= 50);
r_pomcp = simulate(rollout_sim, m, planner, belief_updater);

print(r_pomcp)

-2.0301359596821476

In [9]:
# find the Mean Total Reward for our policy
using Statistics

total_rewards = []

for exp = 1:5
    println(string(exp))
    
    Random.seed!(exp)
    
    # p = ToEnd(0)
    #print(step.r for step in stepthrough(m,p,belief_updater, max_steps=100))
    traj_rewards = sum([step.r for step in stepthrough(m, planner, belief_updater, max_steps=100)])
    
    push!(total_rewards, traj_rewards)
end

@printf("Mean Total Reward: %.3f, StdErr Total Reward: %.3f", mean(total_rewards), std(total_rewards)/sqrt(5))

1
2
3
4
5
Mean Total Reward: -9.500, StdErr Total Reward: 3.851

In [11]:
total_rewards

5-element Array{Any,1}:
 -14.0               
   1.3000000000000016
 -14.0               
  -2.0999999999999996
 -18.699999999999996 